# Development of functions to: 1. Find retro steps for input smiles target. 2. Exctract info needed to populate DB models 3. Extract robotic action steps and populate DB

In [1]:
from rdkit import Chem
from rxn4chemistry import RXN4ChemistryWrapper
import os
import time

In [2]:
# Setup IBM RxN API
api_key=os.environ['IBM_API_KEY'] 
rxn4chemistry_wrapper = RXN4ChemistryWrapper(api_key=api_key)
rxn4chemistry_wrapper.create_project('Test actions')

{'response': {'payload': {'id': '5fb7acaba965f9000133a783',
   'metadata': {},
   'embed': {},
   'computedFields': {},
   'createdOn': 1605872811326,
   'createdBy': '5421668d-2db4-4064-a1e6-f77e33938145',
   'modifiedOn': 1605872811326,
   'modifiedBy': '5421668d-2db4-4064-a1e6-f77e33938145',
   'name': 'Test actions',
   'description': None,
   'attempts': [],
   'visibility': None},
  'metadata': {'uiMessages': {'errors': [], 'infos': [], 'warnings': []},
   'extendedPagination': {}}}}

In [ ]:
class Reaction(object):
    
    def __init__(self, targetsmiles):
        self.smiles = targetsmiles
    
    def getRetroSyn(self):
        # NB need to add method_id to function call via views and urls -
        # see - https://stackoverflow.com/questions/59842406/how-to-pass-an-id-to-a-view-as-an-argument-in-django
        """
        Use the IBM API to get some possible retrosynthesis routes
        """
        # Create dummy dictionary to create while loop to catch when status is a SUCCESS
        results = {}
        results['status'] = None
        reactions = None

        while reactions is None:  
            try:
                time.sleep(30)
                response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product=self.smiles)
                while results['status'] != 'SUCCESS': 
                    time.sleep(30)
                    results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(response['prediction_id'])
                    reactions = results
            except Exception as e:
                print(e)

        return reactions
    
    def saveImage(self):
        # Function to upload product image

    def createReactionModel(self):
        # Function that takes in all the info and creates a reaction object

        # Create Reaction object
        reaction = Reaction()
        # Need the method_id which is passed in as obj in argument
        # Need to have uploaded svg image to productimages folder
        method_obj = Method.objects.get(id=method_id)
        reaction.method_id = method_obj
        reaction.name = name
        reaction.productsmiles = product_smiles
        reaction.productimage = 'productimages/' + product_image_fn
        reaction.productname = product_name
        reaction.save()

# def getRoboActions(reactions):
#     # Each retrosynthetic path predicted has a unique sequence_id that can
#     # be used to create a new synthesis
#     response = rxn4chemistry_wrapper.create_synthesis_from_sequence(
#         sequence_id=results['retrosynthetic_paths'][0]['sequence_id'])
#     print(response['synthesis_id'])

# # get the entire list of actions for the entire synthesis, as well as a tree representation
# synthesis_tree, ordered_tree_nodes, ordered_list_of_actions = rxn4chemistry_wrapper.get_synthesis_plan(
#     synthesis_id=response['synthesis_id']
# )
# for action in ordered_list_of_actions:
#     print(action)

In [ ]:
reactions = getRetroSyn('CC(C)C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](CC1CCNC1=O)C(O)S(=O)(=O)O')

In [ ]:
reactions['retrosynthetic_paths'][0]

In [ ]:
reactions['retrosynthetic_paths'][0]['children']

In [ ]:
# Each retrosynthetic path predicted has a unique sequence_id that can
# be used to create a new synthesis
response = rxn4chemistry_wrapper.create_synthesis_from_sequence(
    sequence_id=reactions['retrosynthetic_paths'][0]['sequenceId'])
print(response['synthesis_id'])

# get the entire list of actions for the entire synthesis, as well as a tree representation
synthesis_tree, ordered_tree_nodes, ordered_list_of_actions = rxn4chemistry_wrapper.get_synthesis_plan(
    synthesis_id=response['synthesis_id'])
for action in ordered_list_of_actions:
    print(action)

In [ ]:
# Need to convert names to smiles!
# Use the awesome Cactus API - https://cactus.nci.nih.gov/

from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(name):
    try:
        name_converted = quote(name)
        url= 'https://cactus.nci.nih.gov/chemical/structure/' + name_converted + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'

identifiers  = ['Z-Leu-OSu']

for ids in identifiers :
    print(ids, CIRconvert(ids))